In [1]:
#Used Python 3.5.2

# Importing libraries
%matplotlib inline
from IPython.display import display
# library for plotting
import matplotlib.pyplot as plt
# tensorflow for machine learning
import tensorflow as tf
#numpy for array
import numpy as np
import PIL.Image 
from PIL import Image
# skimage image processing
from skimage import io,transform,img_as_float
# for reading and saving
from skimage.io import imread,imsave
# for Returning a 2-D array with ones on the diagonal and zeros elsewhere
from numpy import eye 

In [2]:
import scipy

In [3]:
# this function takes the source image and put the source image color on target image
#without loosing any target image content. 
# So we can use for preserving the color of visual-art or add any color which we like
#using source color image
#This function uses the linear color transfer techniques and par tof the code from
#https://github.com/ProGamerGov/Neural-Tools/blob/master/linear-color-transfer.py
#referece from reserach paper https://arxiv.org/pdf/1606.05897.pdf

def color_tranfer(target_img,source_img):
    #loadng the source image as numpy array and do the normalization
    source_img = np.float32(Image.open(source_img))/256
    #loadng the target image as numpy array and do the normalization
    target_img = np.float32(target_img)/256
    
    def change_color(target_img, source_img, eps=1e-5):
    
        #change the colour distribution of the target image to that of the source image
        #using a linear transform
        #getting mu of target image
        mu_of_target = target_img.mean(0).mean(0)
        #taking the difference of target image and mu of target
        t = target_img - mu_of_target
        #doing the transpose and reshape
        t = t.transpose(2,0,1).reshape(3,-1)
        Ct = t.dot(t.T) / t.shape[1] + eps * eye(t.shape[0])
        #getting mu of target image
        mu_of_source = source_img.mean(0).mean(0)
        #taking the difference of target image and mu of target
        s = source_img - mu_of_source
        s = s.transpose(2,0,1).reshape(3,-1)
        Cs = s.dot(s.T) / s.shape[1] + eps * eye(s.shape[0])
        # applying pca technique
        eva_t, eve_t = np.linalg.eigh(Ct)
        Qt = eve_t.dot(np.sqrt(np.diag(eva_t))).dot(eve_t.T)
        eva_s, eve_s = np.linalg.eigh(Cs)
        Qs = eve_s.dot(np.sqrt(np.diag(eva_s))).dot(eve_s.T)
        ts = Qs.dot(np.linalg.inv(Qt)).dot(t)
        
        matched_img = ts.reshape(*target_img.transpose(2,0,1).shape).transpose(1,2,0)
        matched_img += mu_of_source
        matched_img[matched_img>1] = 1
        matched_img[matched_img<0] = 0
        return matched_img
    #Using the change_color function to get the output image
    output_img = change_color(target_img, source_img, eps=float(1e-5))
    imsave('output_name.png', output_img)
    #returning the output image
    return output_img

In [4]:
#donwload the VGG16 model from here:http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-16.mat 
#store under ./data
#We are now going to use the pretrained VGG16 model 
import vgg16
vgg16.data_dir = 'vgg16/'
# download model in vgg16 folder
vgg16.maybe_download()

Data has apparently already been downloaded and unpacked.


In [5]:
# make sure the VGG16 model is stored in VGG_PATH
VGG_PATH = 'data/imagenet-vgg-verydeep-16.mat'

In [6]:
#function to load image and resize it
def load_image(filename, max_size=None):
    #opening the image
    image = Image.open(filename)
    if max_size is not None:
        # find the rescale-factor
        # keeping the proportion between height and width.
        fact = max_size / np.max(image.size)
        # Scale the image's height and width
        size = np.array(image.size) * fact
        # The size is now floating-point because it was scaled.
        # PIL requires the size to be integers.
        size = size.astype(int)
        # Resize the image
        image = image.resize(size, Image.LANCZOS)
    # Convert to numpy floating-point array.
    return np.float32(image)

In [7]:
#function to save image as a jpeg for image as numpy array, pixel between 0 to 255
def save_image(image, filename):
    # cliping the the pixel-values to have in 0 and 255
    image = np.clip(image, 0.0, 255.0)
    # Conversion to bytes
    image = image.astype(np.uint8)
    # Write the file in jpeg
    with open(filename, 'wb') as file:
        Image.fromarray(image).save(file, 'jpeg')

In [8]:
#function to plot image as a jpeg for image as numpy array, pixel between 0 to 255
def plot_image_big(image):
    # cliping the the pixel-values to have in 0 and 255
    image = np.clip(image, 0.0, 255.0)
    # Conversion to bytes
    image = image.astype(np.uint8)
    # Converting to a PIL-image and display
    display(Image.fromarray(image))

In [9]:
# function to plot the tyle image, input image, and final image
def plot_images(input_image, style_image, final_image):
    # Create figure with sub-plots
    fig, axes = plt.subplots(1, 3, figsize=(10, 10))
    # Adjust vertical spacing
    fig.subplots_adjust(hspace=0.1, wspace=0.1)
    # Use interpolation to smooth pixels
    # Interpolation 
    interpolation = 'sinc'
    # Plot the style-image
    ax = axes.flat[0]
    ax.imshow(style_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Style Image")
    
    # Plot the Input-image.
    # normalizing in [0.0, 1.0] range by dividing with 255
    ax = axes.flat[1]
    ax.imshow(input_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Input Image")

    # Plot the final image.
    ax = axes.flat[2]
    ax.imshow(final_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Final Image")

    # Remove ticks from all the plots.
    for ax in axes.flat:
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [10]:
#loss function for optimization 
#calculating the mean square error for a and b
def mean_squared_error(a, b):
    return tf.reduce_mean(tf.square(a - b))
#the loss-function for the input image is the Mean Squared Error of the feature 
#activations in given layers in model between the imput image and the 
#final image. When this content loss is minimized, it therefore means
#that the final image has feature activations in the given layers that
#are very similar to the activations of the input image. Depending on 
#which layers, it will transfer the contours from the 
#input image to the finalimage

In [11]:
 #loss-function for the input image.
def create_content_loss(session, model, content_image, layer_ids):
    # feed-dict with the content-image.
    feed_dict = model.create_feed_dict(image=content_image)
    # references to  tensors for layers
    layers = model.get_layer_tensors(layer_ids)
    # Calculate the output values of those layers when
    # feeding the content-image to the model
    values = session.run(layers, feed_dict=feed_dict)
    # Set the model's graph as the default so we can add
    # computational nodes to it
    with model.graph.as_default():
        # Initialize layer_loss function
        layer_losses = []
        # each layer and its  values for the content-image
        for value, layer in zip(values, layers):
            # These are the values that are calculated
            # for this layer in the model when inputting
            # the content-image.it is a const
            value_const = tf.constant(value)
            # The loss function for layer is the
            # Mean Squared Error between the layer-values
            loss = mean_squared_error(layer, value_const)
            # Add the loss-function for this layer to list of loss-functions
            layer_losses.append(loss)
        # The combined loss for all layers is average
        total_loss = tf.reduce_mean(layer_losses)    
    return total_loss

In [12]:
#we like to measure which features in the style layers activate simultaneously 
#for the style image, and then copy this activation pattern to the final image
#We use Gram matrix for the tensors output by the style layers. 
#The Gram matrix is a matrix of dot products for the vectors of the feature
#activations of a style layer.For if an entry in the Gram matrix has a large value, 
#then it means the two features do activate simultaneously for the given style-image.
def gram_matrix(tensor):
    shape = tensor.get_shape()
    # feature channels for the input tensor,from a convolutional layer with 4-dim
    num_channels = int(shape[3])
    # Reshape the tensor so it is a 2-dim matrix, means doing flatening
    matrix = tf.reshape(tensor, shape=[-1, num_channels])
    # getting dot-products of all combinations of the feature-channels as gram
    gram = tf.matmul(tf.transpose(matrix), matrix)
    return gram                            

In [13]:
#the loss-function for the style-image.Here we calculate the Mean Squared Error 
#for the Gram-matrices 
def create_style_loss(session, model, style_image, layer_ids):
    # Create a feed-dict with the style-image.
    feed_dict = model.create_feed_dict(image=style_image)
    # references to the tensors for the given layers.
    layers = model.get_layer_tensors(layer_ids)
    # Set the model's graph as the default so we can add
    # computational nodes to it. 
    with model.graph.as_default():
        # calculating the Gram-matrices for each of the layers.
        gram_layers = [gram_matrix(layer) for layer in layers]
        # Calculate the values of Gram-matrices when feeding the style-image 
        values = session.run(gram_layers, feed_dict=feed_dict)
        # Initialize loss-functions
        layer_losses = []
        # For each Gram-matrix layer and its corresponding values
        for value, gram_layer in zip(values, gram_layers):
            # Gram-matrix values that calculated for this layer when inputting style image
            value_const = tf.constant(value)

            # loss-function for this layer: Mean Squared Error between the Gram-matrix values
            # for the content and final image
            loss = mean_squared_error(gram_layer, value_const)

            # Add the loss-function list of loss-functions
            layer_losses.append(loss)
        # The combined loss for all layers 
        total_loss = tf.reduce_mean(layer_losses)
    return total_loss

In [14]:
#loss-function for denoising the final image. The algorithm used
#Total Variation Denoising(https://en.wikipedia.org/wiki/Total_variation_denoising) 
#and  it shifts the image one pixel in the x and y axis, calculates the difference from 
#the original image, takes the absolute value to the difference is a positive number, 
#and sums over all the pixels in the image. This loss-function suppress some of the 
#noise in the image
def create_denoise_loss(model):
    loss = tf.reduce_sum(tf.abs(model.input[:,1:,:,:] - model.input[:,:-1,:,:])) + \
           tf.reduce_sum(tf.abs(model.input[:,:,1:,:] - model.input[:,:,:-1,:]))

    return loss

In [17]:
# final function for gradient descent to find an image that minimizes the
#loss functions of the content layers and style layers. From here, final image
#will take contours of the input image, and resembles the colours and textures
#of the style-image.
def style_transfer(content_image, style_image,
                   content_layer_ids, style_layer_ids,
                   weight_content=1.5, weight_style=10.0,
                   weight_denoise=0.3,
                   num_iterations=120, step_size=10.0,preserve_color='no'):

    # instance of the VGG16-model
    vmodel = vgg16.VGG16()
    # getting weights and mean pixel form model
    vgg_weights, vgg_mean_pixel = vmodel.load_net(VGG_PATH)
    # starting tensorflow session 
    session = tf.InteractiveSession(graph=vmodel.graph)
    # writing graph to tensorboard
    writer = tf.summary.FileWriter('./tensorboard')
    writer.add_graph(session.graph)
    # Print the names of the content-layers.
    print("Input layers:")
    print(vmodel.get_layer_names(content_layer_ids))
    print()
    # Print the names of the style-layers.
    print("Style layers:")
    print(vmodel.get_layer_names(style_layer_ids))
    print()
    # loss-function for the content-layers and -image
    loss_content = create_content_loss(session=session,model=vmodel,content_image=content_image,layer_ids=content_layer_ids)
    # loss-function for the style-layers and -image.
    loss_style = create_style_loss(session=session,model=vmodel,style_image=style_image,layer_ids=style_layer_ids)    
    # Create the loss-function for the denoising of final image.
    loss_denoise = create_denoise_loss(vmodel)
    # TensorFlow variables for adjusting the values of loss-functions.
    adj_content = tf.Variable(1e-10, name='adj_content')
    adj_style = tf.Variable(1e-10, name='adj_style')
    adj_denoise = tf.Variable(1e-10, name='adj_denoise')
    # Initialize the adjustment values for the loss-functions
    session.run([adj_content.initializer,adj_style.initializer,adj_denoise.initializer])

    # updating the adjustment values. 
    #small value 1e-10 added to avoid the possibility of division by zero.
    update_adj_content = adj_content.assign(1.0 / (loss_content + 1e-10))
    update_adj_style = adj_style.assign(1.0 / (loss_style + 1e-10))
    update_adj_denoise = adj_denoise.assign(1.0 / (loss_denoise + 1e-10))

    # This is the weighted loss-function that we will minimize
    # below in order to generate the mixed-image.
    loss_combined = weight_content * adj_content * loss_content + weight_style * adj_style * loss_style + weight_denoise * adj_denoise * loss_denoise
    # mathematical function for the gradient of the combined loss-function 
    gradient = tf.gradients(loss_combined, vmodel.input)
    # tensors list to run in each optimization iteration
    run_list = [gradient, update_adj_content, update_adj_style,update_adj_denoise]
    # The mixed/inputimage is initialized with random noise.
    mixed_image = np.random.rand(*content_image.shape) + 128
    for i in range(num_iterations):
        # feed-dict with the mixed/input-image
        feed_dict = vmodel.create_feed_dict(image=mixed_image)
        # calculate the value of gradient
        [grad, adj_content_val, adj_style_val, adj_denoise_val] = session.run(run_list, feed_dict=feed_dict)
        # Reduce the dimensionality of  gradient
        grad = np.squeeze(grad)
        # Scale the step-size to the gradient-values
        step_size_scaled = step_size / (np.std(grad) + 1e-8)
        # Update the image by following the gradient.
        mixed_image -= grad * step_size_scaled
        # clip image to have valid pixel-values between 0 and 255
        mixed_image = np.clip(mixed_image, 0.0, 255.0)
        # Display status every 10 iterations
        if (i % 100 == 0) or (i == num_iterations - 1):
            print()
            print("Iteration:", i)
            # Print adjustment weights for loss-functions
            msg = "Weight Adj. for Input: {0:.2e}, Style: {1:.2e}, Denoise: {2:.2e}"
            print(msg.format(adj_content_val, adj_style_val, adj_denoise_val))
            # to preserve input image color or not
            if preserve_color.upper()==('YES'):
                output_img1=color_tranfer(mixed_image,content_filename)
                # plotting the final image, input image and style image
                plot_images(input_image=content_image,style_image=style_image,final_image=output_img1*255)
            else:
                output_img1=mixed_image
                # plotting the final image, input image and style image
                plot_images(input_image=content_image,style_image=style_image,final_image=output_img1)
            
    print()
    print("Final image:")
    if preserve_color.upper()==('YES'):
        plot_image_big(output_img1*255)
    else:
        plot_image_big(output_img1)
    # Close the TensorFlow session
    session.close()
    #return final_image
    return output_img1

In [18]:
#store input image in ./images folder and load input image
content_filename = 'images/input_image.jpg'
content_image = load_image(content_filename, max_size=None)
#store style image in ./images folder and load input image
style_filename = 'images/style_image1.jpg'
style_image = load_image(style_filename, max_size=300)

In [ ]:
#In VGG model we have different layers, so I am using 5th layer with index 4
content_layer_ids = [4]
#VGG has 13 convolutional layers so I am selecting all of them
style_layer_ids = list(range(13))

In [ ]:
# lets see the results by running functions, we are also givings weights of content,style and denoise.
#it will take more time on CPU than GPU depending on iterations
# this function let you decide to preserve color or not
img = style_transfer(content_image=content_image,style_image=style_image,content_layer_ids=content_layer_ids,
                     style_layer_ids=style_layer_ids,weight_content=10.5,weight_style=5.5,weight_denoise=0.3,
                     num_iterations=400,step_size=10.0,preserve_color='yes')

In [ ]:
#store input image in ./images folder and load input image
content_filename = 'images/MJ.jpg'
content_image = load_image(content_filename, max_size=None)
#store style image in ./images folder and load input image
style_filename = 'images/style_3.jpg'
style_image = load_image(style_filename, max_size=300)

In [ ]:
# lets see the results by running functions, we are also givings weights of content,style and denoise.
#it will take more time on CPU than GPU depending on iterations
# this function let you decide to preserve color or not
img = style_transfer(content_image=content_image,style_image=style_image,content_layer_ids=content_layer_ids,
                     style_layer_ids=style_layer_ids,weight_content=10.5,weight_style=5.5,weight_denoise=0.3,
                     num_iterations=100,step_size=10.0,preserve_color='yes')

In [ ]:
# lets see the results by running functions, we are also givings weights of content,style and denoise.
#it will take more time on CPU than GPU depending on iterations
# this function let you decide to preserve color or not
img = style_transfer(content_image=content_image,style_image=style_image,content_layer_ids=content_layer_ids,
                     style_layer_ids=style_layer_ids,weight_content=10.5,weight_style=5.5,weight_denoise=0.3,
                     num_iterations=100,step_size=10.0,preserve_color='no')

In [ ]:
#store input image in ./images folder and load input image
content_filename = 'images/bright.jpg'
content_image = load_image(content_filename, max_size=None)
#store style image in ./images folder and load input image
style_filename = 'images/style_3.jpg'
style_image = load_image(style_filename, max_size=300)

In [ ]:
# lets see the results by running functions, we are also givings weights of content,style and denoise.
#it will take more time on CPU than GPU depending on iterations
# this function let you decide to preserve color or not
img = style_transfer(content_image=content_image,style_image=style_image,content_layer_ids=content_layer_ids,
                     style_layer_ids=style_layer_ids,weight_content=8.5,weight_style=10.5,weight_denoise=0.3,
                     num_iterations=100,step_size=10.0
                     ,preserve_color='yes')

In [ ]:
# lets see the results by running functions, we are also givings weights of content,style and denoise.
#it will take more time on CPU than GPU depending on iterations
# this function let you decide to preserve color or not
img = style_transfer(content_image=content_image,style_image=style_image,content_layer_ids=content_layer_ids,
                     style_layer_ids=style_layer_ids,weight_content=8.5,weight_style=10.5,weight_denoise=0.3,
                     num_iterations=100,step_size=10.0
                     ,preserve_color='no')

In [ ]:
#store input image in ./images folder and load input image
content_filename = 'images/bright.jpg'
content_image = load_image(content_filename, max_size=None)
#store style image in ./images folder and load input image
style_filename = 'images/dark_style.jpg'
style_image = load_image(style_filename, max_size=300)

In [ ]:
# lets see the results by running functions, we are also givings weights of content,style and denoise.
#it will take more time on CPU than GPU depending on iterations
# this function let you decide to preserve color or not
img = style_transfer(content_image=content_image,style_image=style_image,content_layer_ids=content_layer_ids,
                     style_layer_ids=style_layer_ids,weight_content=10.5,weight_style=2.5,weight_denoise=0.3,
                     num_iterations=100,step_size=10.0
                     ,preserve_color='no')